# Kmeans1

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class KMeans:
    def __init__(self, k=2, epochs=100, ep=1e-4, seed=42):
        self.k = k
        self.ep = ep
        self.epochs = epochs
        np.random.seed(seed)
        self.centroids = None
        self.labels = None
        self.j_hist = []

    def init_centroids(self, X):
        idxs = np.random.choice(X.shape[0], self.k, replace=False)
        return X[idxs]
    
    def assign_cluster(self, X, centroids):
        assign = []
        for x in X:
            distances = [self._dist(x, pt) for pt in centroids]
            assign.append(np.argmin(distances))
        return np.array(assign)
    
    def _dist(self, a, b):
        return np.sqrt(np.sum((a - b) ** 2))
    
    def update_centroids(self, X, labels):
        centroids = np.zeros((self.k, X.shape[1]))
        for i in range(self.k):
            points = X[labels == i]
            if len(points) > 0:
                centroids[i] = points.mean(axis=0)
            else:
                centroids[i] = X[np.random.choice(X.shape[0])]
        return centroids
    
    def compute_objective(self, X, labels, centroids):
        total = 0
        for i in range(self.k):
            cluster_points = X[labels == i]
            total += np.sum((cluster_points - centroids[i]) ** 2)

        return total / X.shape[0]
    
    def fit(self, X):
        self.centroids = self.init_centroids(X)
        for i in range(self.epochs):
            labels = self.assign_cluster(X, self.centroids)
            new_centroids = self.update_centroids(X, labels)
            j = self.compute_objective(X, labels, new_centroids)
            self.j_hist.append(round(j, 4))
            if np.linalg.norm(new_centroids - self.centroids) < self.ep:
                break
            self.centroids = new_centroids
        self.labels = labels
        return self
    
    def plot_convergence(self):
        plt.figure()
        plt.plot(range(1, len(self.j_hist) + 1), self.j_hist, marker='o')
        plt.title('Convergece of KMeans Objective j')
        plt.xlabel('epochs')
        plt.ylabel('J (Average Squared Distance)')
        plt.grid(True)
        plt.show()

    def plot_clusters(self, X):
        plt.figure()
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'cyan']
        for i in range(self.k):
            pts = X[self.labels == i]
            plt.scatter(pts[:, 0], pts[:, 1], s=50, c=colors[i % len(colors)], label=f"Cluster {i + 1}")
        plt.scatter(self.centroids[:, 0], self.centroids[:, 1], c='black', s=150, marker='x', label='centroids')
        plt.title('K Means clustering Result')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.legend()
        plt.show()

# DBSCAN

In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class DBSCAN:
    def __init__(self, eps=3, minPts=2):
        self.eps = eps
        self.minPts = minPts

    def fit(self, X):
        self.X = X
        n = len(X)
        self.visited = np.zeros(n, dtype=bool)
        self.cluster_labels = np.zeros(n, dtype=int)
        self.core_points = []
        self.cluster_id = 0

        self.dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                self.dist_matrix[i, j] = np.linalg.norm(X[i] - X[j])
        self.dist_matrix = np.round(self.dist_matrix, 2)

        self.neighbors = []
        for i in range(n):
            neigh = np.where(self.dist_matrix[i] <= self.eps)[0]
            self.neighbors.append(neigh)
            if len(neigh) >= self.minPts:
                self.core_points.append(i)

        for i in range(n):
            if not self.visited[i] and i in self.core_points:
                self.cluster_id += 1
                self.expand_cluster(i)

    def expand_cluster(self, i):
        queue = [i]
        while queue:
            point = queue.pop(0)
            if not self.visited[point]:
                self.visited[point] = True
                self.cluster_labels[point] = self.cluster_id
                if point in self.core_points:
                    for neigh in self.neighbors[point]:
                        if self.cluster_labels[neigh] == 0:
                            queue.append(neigh)

    def noise_ratio(self):
        noise_count = np.sum(self.cluster_labels == 0)
        return noise_count / len(self.X)
    
    def print_results(self):
        print("Core points:", ' '.join(str(i+1) for i in self.core_points))
        print("Cluster Assignments:", ' '.join(str(lbl) for lbl in self.cluster_labels))
        print("Noise Ratio =", round(self.noise_ratio(), 2))

    def plot_clusters(self):
        plt.figure(figsize=(6, 5))
        colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan']
        for idx, label in enumerate(set(self.cluster_labels)):
            if label == 0:
                pts = self.X[self.cluster_labels == label]
                plt.scatter(pts[:, 0], pts[:, 1], marker='x', color='k', label='Noise')
            else:
                pts = self.X[self.cluster_labels == label]
                plt.scatter(pts[:, 0], pts[:, 1], color=colors[label % len(colors)], label=f'Cluster {label}')
        plt.title(f'DBSCAN Clustering (eps={self.eps}, minPts={self.minPts})')
        plt.xlabel('x1')
        plt.ylabel('x2')
        plt.legend()
        plt.show()

# Decision Tree

In [9]:
import numpy as np

In [10]:
def entropy(y):
    classes, counts = np.unique(y, return_counts=True)
    ps = counts / len(y)
    return -np.sum(ps * np.log2(ps + 1e-9))

In [11]:
def gini(y):
    classes, counts = np.unique(y, return_counts=True)
    ps = counts / len(y)
    return 1 - np.sum(ps ** 2)

In [12]:
def best_split(X, y, impurity_fn):
    best_gain = -np.inf
    best_thres = None
    best_feature = None
    m, n = X.shape

    for f in range(n):
        si = np.argsort(X[:, f])
        s_X = X[si, f]
        s_y = y[si]

        thresholds = (s_X[:-1] + s_X[1:]) / 2
        for t in thresholds:
            left_mask = X[:, f] <= t
            right_mask = ~left_mask
            left_y = y[left_mask]
            right_y = y[right_mask]

            if len(left_y) == 0 or len(right_y) == 0:
                continue

            impl = impurity_fn(left_y)
            impr = impurity_fn(right_y)

            wimp = (len(left_y) / m) * impl + (len(right_y) / m) * impr
            gain = impurity_fn(y) - wimp

            if gain > best_gain:
                best_gain = gain
                best_thres = t
                best_feature = f

    return best_feature, best_thres

In [13]:
def train(X, y, depth=0, max_depth=None, min_samples_leaf=1, impurity_fn=entropy):
    n_samples = len(y)
    if depth >= max_depth or n_samples <= min_samples_leaf or len(np.unique(y)) == 1:
        return np.bincount(y).argmax()
    
    f, t = best_split(X, y, impurity_fn)
    if f is None:
        return np.bincount(y).argmax()
    
    lm = X[:, f] <= t
    rm = ~lm
    
    ltree = train(X[lm], y[lm], depth + 1, max_depth, min_samples_leaf, impurity_fn)
    rtree = train(X[rm], y[rm], depth + 1, max_depth, min_samples_leaf, impurity_fn)

    return {'feature': f, 'threshold': t, 'left': ltree, 'right': rtree}

In [14]:
def predict(tree, sample):
    if isinstance(tree, dict):
        if sample[tree['feature']] <= tree['threhold']:
            return predict(tree['left'], sample)
        else:
            return predict(tree['right'], sample)
    else:
        return tree

In [15]:
def evaluate(X, y, tree):
    pred = [predict(tree, sample) for sample in X]
    return np.mean(np.array(pred) == y)

In [16]:
def grid_search(X_train, y_train, X_val, y_val, depths, min_samples, impurity_fn):
    best_depth = None
    best_min_samples = None
    best_acc = -np.inf
    best_tree = None

    for d in depths:
        for m in min_samples:
            tree = train(X_train, y_train, max_depth=d, min_samples_leaf=m, impurity_fn=impurity_fn)
            val_acc = evaluate(X_val, y_val, tree)
            if val_acc > best_acc:
                best_acc = val_acc
                best_depth = d
                best_min_samples = m
                best_tree = tree

    return best_tree, best_depth, best_min_samples, best_acc

In [17]:
def manual_split(X, y):
    n = len(X)
    n_train = int(0.7 * n)
    n_val = int(0.15 * n)
    n_test = n - n_train - n_val

    X_train = X[:n_train]
    y_train = y[:n_train]

    X_val = X[n_train:n_train + n_val]
    y_val = y[n_train:n_train + n_val]

    X_test = X[n_train + n_val:]
    y_test = y[n_train + n_val:]

    return X_train, y_train, X_val, y_val, X_test, y_test

# Kmeans 2

In [18]:
import numpy as np
import matplotlib.pyplot as plt

In [19]:
def kmeans_plus_plus_init(X, k, seed=0):
    np.random.seed(seed)
    n_samples, _ = X.shape
    centroids = np.zeros((k, X.shape[1]))
    centroids[0] = X[np.random.choice(n_samples)]
    closest = np.full(n_samples, np.inf)

    for id in range(1, k):
        dist_centroid = np.linalg.norm(X - centroids[id - 1], axis=1) ** 2
        closest = np.minimum(closest, dist_centroid)
        probs = closest / closest.sum()
        cumu_probs = np.cumsum(probs)
        r = np.random.rand()
        next_idx = np.searchsorted(cumu_probs, r)
        centroids[id] = X[next_idx]

    return centroids

In [20]:
def assign_clusters(X, centroids):
    distances = np.linalg.norm(X[:, None] - centroids[None, :], axis=2)
    return np.argmin(distances, axis=1)

In [21]:
def update_centroids(X, labels, k):
    n = X.shape[1]
    centroids = np.zeros((k, n))
    for i in range(n):
        cluster_points = X[labels == i]
        if len(cluster_points) > 0:
            centroids[i] = cluster_points.mean(axis=0)
        else:
            centroids[i] = X[np.random.choice(len(X))]

    return centroids

In [22]:
def compute_wcss(X, labels, centroids):
    total = 0.0
    for k in range(len(centroids)):
        cluster_points = X[labels == k]
        dists = np.linalg.norm(cluster_points - centroids[k], axis=1) ** 2
        total += dists.sum()
    return total

In [23]:
def compute_j(X, labels, centroids):
    wcss = compute_wcss(X, labels, centroids)
    return wcss / X.shape[0]

In [24]:
def kmeans(X, k, epochs, seed=0):
    centroids = kmeans_plus_plus_init(X, k, seed)
    for i in range(epochs):
        labels = assign_clusters(X, centroids)
        new_centroids = update_centroids(X, labels, k)
        if np.allclose(new_centroids, centroids):
            break
        centroids = new_centroids
    labels = assign_clusters(X, centroids)
    return centroids, labels

In [25]:
def kneedle_elbow(kmax, wcss_list):
    x1, y1 = 1, wcss_list[0]
    x2, y2 = kmax, wcss_list[-1]

    max_dist = -1
    elbow = 1

    for k in range(1, kmax + 1):
        x0 = k
        y0 = wcss_list[k - 1]
        numerator = abs((x2 - x1) * (y1 - y0) - (x1 - x0)*(y2 - y1))
        denominator = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2) 
        dist = numerator / denominator
        if dist > max_dist:
            max_dist = dist
            eblow = k
    return elbow